## 4장 실제 데이터를 텐서로 표현해보기

### 4.1 이미지 다루기

In [1]:
# 일반 이미지의 각 픽셀의 스칼라 값은 8비트 정수 인코딩을 사용
# 의학이나 과학 혹은 군사용 애플리케이션에서는 12비트나 16비트처럼 높은 정밀도를 사용
# 높은 정밀도 --> 뼈의 밀도, 온도, 심도 등 물리적 속성 정보를 픽셀에 인코딩 시 더 넓은 범위나 높은 민감도를 가지도록 생성 가능

In [2]:
import numpy as np
import torch
import warnings
warnings.filterwarnings('ignore')
torch.set_printoptions(edgeitems=2, threshold=50) # edgeitems: 각 차원의 시작과 끝에 요약된 배열 항목의 수 (default=3)

In [3]:
import imageio

img_arr = imageio.imread('./data/image-dog/bobby.jpg') # 이미지를 불러와서 numpy 배열 형태로 변환
img_arr.shape # (H * W * C)

# 컬러를 숫자로 인코딩 --> RGB의 강도를 나타내는 세 개의 숫자로 컬러를 정의
# 각 컬러 채널은 흑백 이미지에서 픽셀 단위로 흑백의 강도를 나타내는 값을 가지는 맵
# ex) 붉은 컬러에 대한 이미지 맵을 보는 것은 붉은 선글라스를 끼고 그림을 보는 느낌

# 이진영상(binary image): 1비트 픽셀 --> 검은색은 0, 흰색은 1
# 흑백영상(gray-scale image): 하나의 픽셀을 8비트, 0~255의 숫자로 표현 --> 가장 어두운 것은 0, 가장 밝은 것은 255
# 컬러영상(color image): RGB 각각 8비트, 0~255의 숫자로 표현 --> (255, 255, 255)는 흰색, (0, 0, 0)은 검은색

(720, 1280, 3)

In [4]:
img = torch.from_numpy(img_arr)
out = img.permute(2, 0, 1) # torch 모듈로 이미지를 다루기 위해서는 (C * H * W) 순서의 3차원으로 변환 필요
out.shape

# permute(): 복사본을 만들지 않고 deep copy
# img, out의 저장 공간이 동일 --> img의 픽셀을 변경하면 out의 픽셀도 자동으로 변경

# cf) tensorflow 프레임워크는 채널 차원을 마지막에 배치하는 (H * W * C) 레이아웃을 사용

torch.Size([3, 720, 1280])

In [5]:
batch_size = 3 # 여러 장의 이미지를 batch로 할당
batch = torch.zeros(batch_size, 3, 256, 256, dtype=torch.uint8)
batch.shape

# 텐서를 효율적으로 만들고자 stack을 사용해 미리 0으로 된 적당한 공간을 할당
# (N * C * H * W) --> 이미지 수, RGB, H = W = 256 pixels / uint8: 색상을 8비트 정수로 표현 (0~255)

# 신경망의 입력으로 사용할 여러 장의 이미지 데이터셋을 생성 --> 첫 번째 차원에 여러 이미지를 batch로 삽입
# cf) 하나의 이미지를 다루는 경우: batch = torch.unsqueeze(out, 0) --> 0번 인덱스에 1인 차원 추가

torch.Size([3, 3, 256, 256])

In [6]:
batch = torch.rand(batch_size, 3, 256, 256) # 0~1의 난수로 텐서 공간을 할당하는 방법
batch.size()

torch.Size([3, 3, 256, 256])

In [8]:
import os

data_dir = './data/image-cats/'
filenames = [name for name in os.listdir(data_dir)
             if os.path.splitext(name)[-1] == '.png'] # .png로 끝나는 파일
for i, filename in enumerate(filenames):
    img_arr = imageio.imread(os.path.join(data_dir, filename))
    img_t = torch.from_numpy(img_arr)
    img_t = img_t.permute(2, 0, 1)
    img_t = img_t[:3] # RGB만 유지(투명도 알파까지 있는 경우도 존재) --> 이미 3채널이므로 여기서는 변화 x
    batch[i] = img_t # 0으로 된 공간을 이미지 텐서로 채워넣는 작업

In [9]:
# 데이터 정규화
batch = batch.float() # 부동소수점으로 표현이 가능하도록 8비트 정수(uint8)를 float으로 변환
batch /= 255.0 # 픽셀 값을 부호 없는 8비트 정수의 최댓값인 255로 나누어 정규화 (방법 #1)

# 부동소수점: 실수를 표현할 때 소수점의 위치를 고정하지 않는 것 (ex: 3.14E+16)
# 신경망은 일반적으로 부동소수점 텐서를 입력으로 사용
# 입력 값이 대략 0~1이거나 -1~1일 때 훈련 성능이 가장 우수 --> 빌딩 블록 정의 방식의 영향
# 값의 범위가 0과 1 또는 -1과 1 사이에 놓일 수 있도록 정규화

In [10]:
# 방법 #2: 입력 데이터의 평균과 표준편차를 구하여 평균이 0이고 각 채널 값이 표준편차를 넘지 않도록 정규화
n_channels = batch.shape[1]
for c in range(n_channels):
    mean = torch.mean(batch[:, c])
    std = torch.std(batch[:, c])
    batch[:, c] = (batch[:, c] - mean) / std

# 여기서는 하나의 batch 이미지 묶음에 대한 정규화를 수행
# 실제로 이미지로 작업할 때는 미리 모든 훈련 데이터에 대한 평균과 표준편차를 계산하고,
# 이렇게 사전에 계산되고 고정된 값으로 빼고 나누는 방법이 좋은 관례 --> transforms.Normalize()

# 신경망의 입력에 대해서는 정규화 이외에도 회전, 크기 변환, 자르기 등 기하학적 변환을 추가적으로 수행 가능
# 훈련에 도움이 되는 작업(입력 데이터의 다양성 확보)이거나, 신경망이 요구하는 입력 형태(이미지 크기)를 위해 필요한 작업